In [21]:
import os
import random
import re
import sys

DAMPING = 0.85
SAMPLES = 10000





def crawl(directory):
    """
    Parse a directory of HTML pages and check for links to other pages.
    Return a dictionary where each key is a page, and values are
    a list of all other pages in the corpus that are linked to by the page.
    """
    pages = dict()

    # Extract all links from HTML files
    for filename in os.listdir(directory):
        if not filename.endswith(".html"):
            continue
        with open(os.path.join(directory, filename)) as f:
            contents = f.read()
            links = re.findall(r"<a\s+(?:[^>]*?)href=\"([^\"]*)\"", contents)
            pages[filename] = set(links) - {filename}

    # Only include links to other pages in the corpus
    for filename in pages:
        pages[filename] = set(
            link for link in pages[filename]
            if link in pages
        )

    return pages



In [22]:
if __name__ == "__main__":
    main()

{'ai.html': 1852, 'algorithms.html': 1037, 'c.html': 1287, 'inference.html': 1313, 'logic.html': 258, 'programming.html': 2348, 'python.html': 1212, 'recursion.html': 693}
count = 10000
PageRank Results from Sampling (n = 10000)
  ai.html: 0.1852
  algorithms.html: 0.1037
  c.html: 0.1287
  inference.html: 0.1313
  logic.html: 0.0258
  programming.html: 0.2348
  python.html: 0.1212
  recursion.html: 0.0693
1.0
{'ai.html': 0.125, 'algorithms.html': 0.125, 'c.html': 0.125, 'inference.html': 0.125, 'logic.html': 0.125, 'programming.html': 0.125, 'python.html': 0.125, 'recursion.html': 0.125}
{'ai.html': 0.284375, 'algorithms.html': 0.17812499999999998, 'c.html': 0.17812499999999998, 'inference.html': 0.284375, 'logic.html': 0.125, 'programming.html': 0.33749999999999997, 'python.html': 0.17812499999999998, 'recursion.html': 0.071875}
{'ai.html': 0.284375, 'algorithms.html': 0.17812499999999998, 'c.html': 0.17812499999999998, 'inference.html': 0.284375, 'logic.html': 0.125, 'programming.ht

In [9]:
def transition_model(corpus, page, damping_factor):
    """
    Return a probability distribution over which page to visit next,
    given a current page.

    With probability `damping_factor`, choose a link at random
    linked to by `page`. With probability `1 - damping_factor`, choose
    a link at random chosen from all pages in the corpus.
    """
    
    #init a empty dictionary for distribution
    distribution = {k:0  for k in corpus.keys()}
    
    links_count_in_page = len(corpus[page])
    
    if links_count_in_page ==0:
        for key in corpus:
            distribution[key] = 1/len(corpus)
    else:
        link_probability = damping_factor/len(corpus[page])
        random_prob =  (1-damping_factor)/len(corpus)
        for v in corpus[page]:
            distribution[v] = link_probability + random_prob
    
        for key in corpus:
            if key not in corpus[page]:
                distribution[key] = random_prob
    
    return distribution
    
    
        
def sample_pagerank(corpus, damping_factor, n):
    """
    Return PageRank values for each page by sampling `n` pages
    according to transition model, starting with a page at random.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    corpus_keys = list(corpus.keys())
    page_count_dictionary = {k:0 for k in corpus_keys}
    
    
    #first sample
    page = random.choice(corpus_keys)
    page_count_dictionary[page] +=1
    
    distribution = transition_model(corpus, page, damping_factor)
    
    for i in range(n-1):
        page = random.choices(corpus_keys, weights=list(distribution.values()), k=1)[0]
        distribution = transition_model(corpus, page, damping_factor)
        page_count_dictionary[page] +=1

    print(page_count_dictionary)
    print("count =", sum(page_count_dictionary.values()))
    rank_dictionary = {key:page_count_dictionary[key]/n for key in page_count_dictionary.keys()}
        
    return rank_dictionary
    
    
    
def iterate_pagerank(corpus, damping_factor):
    """
    Return PageRank values for each page by iteratively updating
    PageRank values until convergence.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    N = len(corpus.keys())
    random_walk = (1-damping_factor)/N
    rank_dictionary = {k:(1/N) for k in corpus.keys()}
    rank_dictionary_next = {k:0 for k in corpus.keys()}
    
    not_coverged =True
    while(not_coverged):
        
        for key in corpus.keys():
            pr = random_walk + damping_factor*get_estimate(corpus, key, rank_dictionary)
            rank_dictionary_next[key] = pr
        
        if check_convergence(rank_dictionary, rank_dictionary_next):
            not_coverged = False
        else:
            rank_dictionary = rank_dictionary_next.copy()
        
    return rank_dictionary
    
    

In [14]:
def get_estimate(corpus,key, rank_dictionary):
    estimate =0
    for k, values in corpus.items():
        if k!=key:
            if len(values) ==0:
                for j in corpus.keys():
                    estimate += rank_dictionary[j]/len(corpus.keys())
                
            else:
                if key in values:
                    estimate += rank_dictionary[k]/ len(values)
    return estimate
                
    
def check_convergence(prev_dict, next_dict):
    print(prev_dict)
    print(next_dict)
    flags=[]
    converged = False
    for key in prev_dict:
        diff = prev_dict[key] - next_dict[key]
        if diff <=0.001:
            flags.append(1)
        else:
            flags.append(0)
    
    if sum(flags) == len(prev_dict.keys()):
        converged = True
    
    return converged    
    

In [15]:
a = {"1.html": {"2.html", "3.html"}, "2.html": {"3.html"}, "3.html": {"2.html"}}
b= "3.html"
iterate_pagerank(a, 0.85)

{'1.html': 0.3333333333333333, '2.html': 0.3333333333333333, '3.html': 0.3333333333333333}
{'1.html': 0.05000000000000001, '2.html': 0.475, '3.html': 0.475}
{'1.html': 0.05000000000000001, '2.html': 0.475, '3.html': 0.475}
{'1.html': 0.05000000000000001, '2.html': 0.475, '3.html': 0.475}


{'1.html': 0.05000000000000001, '2.html': 0.475, '3.html': 0.475}